### Revise and understand this notebook 

In [ ]:
## Commenting it since i have already installed these in my virtual environment (or ipynb kernel)
# %pip install chromadb tiktoken 

In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("Key not found")

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

/home/himanshu/miniconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.vectorstores import Chroma

In [5]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [6]:
docs = [doc2, doc2, doc3, doc4, doc5]

In [ ]:
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory='my_chroma_db', # this line will create a directory with specified name
    collection_name='sample'
)

/tmp/ipykernel_17812/615392969.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [ ]:
# add documents
vector_store.add_documents(docs)

['ce7b4180-f1c4-405f-ae00-e4ee3c758f91',
 'c3a80c5c-4174-4c70-b697-e64e1f09a8db',
 'ae13db37-450c-4fa0-8f68-f8278322976e',
 '67f6622d-9952-4685-95e1-4bdf9748832c',
 '3a2dcd7c-a06d-48bf-b9bb-06f01e411aae']

In [17]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['ce7b4180-f1c4-405f-ae00-e4ee3c758f91',
  'c3a80c5c-4174-4c70-b697-e64e1f09a8db',
  'ae13db37-450c-4fa0-8f68-f8278322976e',
  '67f6622d-9952-4685-95e1-4bdf9748832c',
  '3a2dcd7c-a06d-48bf-b9bb-06f01e411aae'],
 'embeddings': array([[-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061],
        [-0.03304848, -0.03280932, -0.02324565, ..., -0.00354889,
         -0.00507506, -0.00217535]]),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  "Rohit Sharma is the most successful captain in IPL hist

In [13]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [14]:
# search with similarity score 
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.8699021339416504),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.9354298114776611)]

In [15]:
# meta data filtering
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    filter={"team": "Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.9354298114776611),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.0301952362060547)]

In [16]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)

In [18]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['ce7b4180-f1c4-405f-ae00-e4ee3c758f91',
  'c3a80c5c-4174-4c70-b697-e64e1f09a8db',
  'ae13db37-450c-4fa0-8f68-f8278322976e',
  '67f6622d-9952-4685-95e1-4bdf9748832c',
  '3a2dcd7c-a06d-48bf-b9bb-06f01e411aae'],
 'embeddings': array([[-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061],
        [-0.03304848, -0.03280932, -0.02324565, ..., -0.00354889,
         -0.00507506, -0.00217535]]),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  "Rohit Sharma is the most successful captain in IPL hist

In [19]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [20]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['ce7b4180-f1c4-405f-ae00-e4ee3c758f91',
  'c3a80c5c-4174-4c70-b697-e64e1f09a8db',
  'ae13db37-450c-4fa0-8f68-f8278322976e',
  '67f6622d-9952-4685-95e1-4bdf9748832c',
  '3a2dcd7c-a06d-48bf-b9bb-06f01e411aae'],
 'embeddings': array([[-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061],
        [-0.03304848, -0.03280932, -0.02324565, ..., -0.00354889,
         -0.00507506, -0.00217535]]),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  "Rohit Sharma is the most successful captain in IPL hist